# Notebook explenation
This notebook is taken for the challenge of the ADS Minor at Fontys Hogeschool ICT. 
<br><br>
The goal of the challenge is to apply what we have learned in the course of the minor.
<br><br>
For the challenge Max de Goede and Wesly Wijnen are working together
<br><br>
# Findings:
## Illegal matches
While exploring the data, an interesting discovery on illegal (games having a higher groupsize than is possible for the matchtype) games was made.
This lead to having only 700 useable matches to trian the network with.
# Table of contents
## 1. Imports
Imports used within the notebook to run it.
## 2. Functions
Functions used within the notebook to run it.
## 3. EDA (Exploratory Data Analysis)
Findings within the data, what have we come up with.
## 4. Data-Preparation
## 5. Modelling
### 5.1 Model without illegal matches (Only contians Solo and SoloFPP matches)
### 5.2 Model with illegal matches (Contains matches with more than the maximum number of groupsize)

# 1. Imports

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 2. Functions

In [ ]:
def get_elo_of_players_in_match(df,matchnr):
    getmatchID = df.matchId[matchnr]
    eloLIstrank  = df.rankPoints[df['matchId'] == getmatchID]
    eloListWin = df.winPoints[df['matchId'] == getmatchID]
    eloLIstrank = np.sort(eloLIstrank)
    eloListWin = np.sort(eloListWin)
    return  eloLIstrank,eloListWin
def create_cat_values(dataframe,column):
    dataframe[column] = dataframe[column].astype('category')
    dataframe[column] = dataframe[column].cat.codes
    column_lenght = len(dataframe[column].unique())
    print(column + ' columnContains:' + str(column_lenght) + ' categories')
    return dataframe

# 3. EDA (Exploratory Data Analysis)/ 4. Data preparation
Lets take a look at what the data looks like and see if we can get any insights from the data!

In [ ]:
df = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv')

In [ ]:
df.columns

There is a wierd rating system where, the player either has rank or win points. It seems that wehn one is used the other is not

In [ ]:
print(get_elo_of_players_in_match(df,20))
get_elo_of_players_in_match(df,21)

In [ ]:
df.rankPoints[df['matchId'] == 'd5f02b7d5ed782']

## Lets create a subset of each match Type. As the results **should** vary pet match type
Here we do 2 things, we create categorical values for the match,group and ID of the player)

In [ ]:
create_cat_values(df,'Id')
create_cat_values(df,'groupId')
create_cat_values(df,'matchId')

In [ ]:
solo = df[df.matchType == 'solo']
duo = df[df.matchType == 'duo']
squad = df[df.matchType == 'squad']
solofpp = df[df.matchType == 'solo-fpp']
duofpp = df[df.matchType == 'duo-fpp']
squadfpp = df[df.matchType == 'squad-fpp']
print('Solo subset size ' + str(len(solo)))
print('Duo subset size ' + str(len(duo)))
print('Squad subset size ' + str(len(squad)))
print('SoloFPP subset size ' + str(len(solofpp)))
print('DuoFPP subset size '+ str(len(duofpp)))
print('SquadFPP subset size '+ str(len(squadfpp)))

### 4.1 Dealing with data inconsistencies
1. Different group size per game.
  - Some squad matches contain, group sizes that exceed 4. There can be multiple reasons for this, we have for now decided to drop all matches containing a groupsize bigger than 4.
  - Same can be said for duo games when it comes to having a maximum of 2 people per group.
  - Same goes for solo where you cant have more than 2 people in the same group.
2. Certain columns will be dropped.
  - Not all features actually add any value, features such as killrank in a game doesn't add much. We can much better use the kills themselves

### 4.1.1 Different group size per game

In [ ]:
squadfpp

In [ ]:
test = squadfpp.groupby(['groupId','matchId']).count()
test

In [ ]:
test[test['Id']> 4]

In [ ]:
test[test['Id']> 30]

In [ ]:
#squadfpp[squadfpp['matchId']== 20999].groupby('groupId').count()
squadfpp[squadfpp['groupId']== 1297474]

In [ ]:
solofirst = solofpp[['Id','groupId']].groupby('groupId').count().max()[0]
solothird = solo[['Id','groupId']].groupby('groupId').count().max()[0]
squadfirst = squadfpp[['Id','groupId']].groupby('groupId').count().max()[0]
squadthird = squad[['Id','groupId']].groupby('groupId').count().max()[0]
duofirst = duofpp[['Id','groupId']].groupby('groupId').count().max()[0]
duothird = duo[['Id','groupId']].groupby('groupId').count().max()[0]
print('Solo max group FP:' + str(solofirst) + " TP:" + str(solothird) )
print('Squad max group FP:' + str(squadfirst) + " TP:" +str(squadthird))
print('Duo max group FP:' + str(duofirst) + " TP:" +str(duothird))

In [ ]:
listofcount = solofpp[['matchId','groupId','Id']].groupby(['groupId','matchId']).count()
listofcount.Id >2 
#listofcount[listofcount['Id'] > solo].index[0][1]

# Illegal matches
Below you can find how we managed to take into account what illegal matches are

In [ ]:
# Creating benchmark of player number
solot = 1
duot = 2
squadt = 4

def get_all_matches(df,maxplayer):
    matcharr = []
    listofcount = df[['matchId','groupId','Id']].groupby(['groupId','matchId']).count()
    listofcount = listofcount[listofcount['Id'] > solot].index
    i = 0
    for items in listofcount:
        matchid = items[1]
        matcharr.append(matchid)
        i = i + 1
    print("Unique number of games: "+str(len(df['matchId'].unique()))+" Unique number of games with more than the acceptable groupnumbers: "+str(len(np.unique(matcharr))))
    matcharr = np.unique(matcharr)
    return matcharr
solofppmatchestodrop = get_all_matches(solofpp,solot)
solomatchestodrop = get_all_matches(solo,solot)
get_all_matches(duofpp,duot)
get_all_matches(duo,duot)
get_all_matches(squadfpp,squadt)
get_all_matches(squad,squadt)

In [ ]:
len(solofppmatchestodrop)

In [ ]:
lenbefore = len(solofpp)
for items in solofppmatchestodrop:
    solofpp = solofpp[solofpp.matchId != items]
lenafter = len(solofpp)
print('Lenght of data before dropping all wrong group measures: ' + str(lenbefore) + " Len after: " + str(lenafter))

lenbefore = len(solo)
for items in solomatchestodrop:
    solo = solo[solo.matchId != items]
lenafter = len(solo)
print('Lenght of data before dropping all wrong group measures: ' + str(lenbefore) + " Len after: " + str(lenafter))

In [ ]:
print(len(solofpp['matchId'].unique()))
len(solo['matchId'].unique())

### Conclusion, 
There are only about 500-600 games with the correct number of group size. If the decision is made to use only games with the correct number of maximum group size. Only the games a few solofpp/solo games will be useable.

### 4.1.2 Dropping unnecesarry columns
Dropping:
 - MatchId
 - group_Id
 - Id
 - Matchtype
 - Num groups
 - maxPlace 
 - killplace
<br><br>
Since these are solo matches we need to drop the following columns too:
 - Revives
 - DBNOs

In [ ]:
solofpp.columns
solofpp = solofpp[['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints','rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc']]
solo.columns
solo = solo[['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints','rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc']]

In [ ]:
solofpp

In [ ]:
def createplots(df):
    coltoplot = ['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc']
    for columns in coltoplot:
        plt.scatter(df.winPlacePerc, df[columns], alpha=0.5)
        plt.title(columns)
        plt.xlabel('Placement')
        plt.ylabel(columns)
        plt.show()

In [ ]:
createplots(solofpp)

Lets now normalize the data

In [ ]:
solofpp.columns

In [ ]:
boxplot = solofpp.boxplot(['winPlacePerc'])

For the first version we will be focussing on the SoloFPP subset, as pefromance per player can only be measured by one person.

In [ ]:
solofpp

We will not use all of the variables, lets see if we can find some cool corrolations

In [ ]:
solofpp.columns
col = ['damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration','winPlacePerc']#,
       #'rideDistance', 'roadKills',
       #'swimDistance', 'vehicleDestroys', 'walkDistance',
       #'weaponsAcquired']

In [ ]:
corrMatrix = solofpp[col].corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
solofpp.nunique()

### Lets create a first network to see what the results can look like, But before this we need to normalize

In [ ]:
from sklearn.model_selection import train_test_split
solofpp = solofpp.dropna()
solofpp.isnull().sum()
y_train = solofpp['winPlacePerc']
X_train = solofpp[['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']]
X_train = X_train[['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
###############################
#           Validate          #
###############################
y_val = solo['winPlacePerc']
X_val = solo[['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
print(solofpp.dtypes)

In [ ]:
X_train.count()

In [ ]:
import matplotlib.pyplot as plt
for items in X_train.columns:
    plt.figure()
    X_train.boxplot([items])


It is important to note that the accuracy does not mean much here as the network is specificly trying to predict the score. And the score can be pretty precise to predict

# 6. Modelling
## 6.1 Model for legal matches

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(300, activation="relu", name="Input_layer"),
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(80, activation="relu"),
        tf.keras.layers.Dense(40, activation="relu"),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(20, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss='mse',
    # List of metrics to monitor
    metrics=['MSE'],
)
history  = model.fit(X_train,y_train,epochs = 200)

In [ ]:
networkoutput = model.predict(X_train)

In [ ]:
prediction = y_train.values

In [ ]:
# Create a dataframe to hold the predicted, actual and difference between the two.
df_to_test_difference = pd.DataFrame(networkoutput,columns = ['Prediction'])
df_to_test_difference['Actual'] = prediction
df_to_test_difference['Difference'] = df_to_test_difference['Actual']- df_to_test_difference['Prediction']

In [ ]:
df_to_test_difference

The best way to evaluate the model is by looking at the mean value that deviates from the prediction

In [ ]:
print("Average deviation: ")
print(df_to_test_difference['Difference'].mean())
print('Maximum deviation: ')
print(df_to_test_difference['Difference'].max())
print('Minimum deviation: ')
print(df_to_test_difference['Difference'].min())

Shows the distribution between the prediction and actual. A perfect straight line would be the ideal scenario.

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(df_to_test_difference.Prediction,df_to_test_difference.Actual)

This graph displays the distribution when we look at the differnce between the prediction and actual number

In [ ]:
ax = sns.boxplot(x=df_to_test_difference["Difference"])

# Test the model
Luckily we had some Solo games that are also useable we will make them ready for usage and run the model over the data and see what kind of results we get!

In [ ]:
val_prediction = model.predict(X_val)
# Create a dataframe to hold the predicted, actual and difference between the two.
df_to_val_difference = pd.DataFrame(val_prediction,columns = ['Prediction'])
df_to_val_difference['Actual'] = y_val.values
df_to_val_difference['Difference'] = df_to_val_difference['Actual']- df_to_val_difference['Prediction']
df_to_val_difference

Lets run the same analysis as we did for the test values

In [ ]:
print("Average deviation: ")
print(df_to_val_difference['Difference'].mean())
print('Maximum deviation: ')
print(df_to_val_difference['Difference'].max())
print('Minimum deviation: ')
print(df_to_val_difference['Difference'].min())

In [ ]:
plt.scatter(df_to_val_difference.Prediction,df_to_val_difference.Actual)

In [ ]:
ax = sns.boxplot(x=df_to_val_difference["Difference"])

### Test the model on the test dataset
Since the compettiion provides a train/test dataset, we will use the test dataset to see what kind of results we can get. Optimally this will be used in the model itself to further improve the model.
#### Lets bring the data through the same pipeline

In [ ]:
colls = ['assists', 'boosts', 'damageDealt',
       'headshotKills', 'heals', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration',
       'rankPoints','rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']
rawtest = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')
tesetdf = rawtest


In [ ]:
solotest = tesetdf[df.matchType == 'solo']
duotest = tesetdf[tesetdf.matchType == 'duo']
squadtest = tesetdf[tesetdf.matchType == 'squad']
solofpptest = tesetdf[tesetdf.matchType == 'solo-fpp']
duofpptest = tesetdf[tesetdf.matchType == 'duo-fpp']
squadfpptest = tesetdf[tesetdf.matchType == 'squad-fpp']

In [ ]:
squadfpp.columns

In [ ]:
solotestmatchestodrop = get_all_matches(solotest,solot)
solofpptestmatchestodrop = get_all_matches(solofpptest,solot)
get_all_matches(duotest,duot)
get_all_matches(duofpptest,duot)
get_all_matches(squadtest,squadt)
get_all_matches(squadfpptest,squadt)

In [ ]:
lenbefore = len(solofpptest)
for items in solofpptestmatchestodrop:
    solofpptest = solofpptest[solofpptest.matchId != items]
lenafter = len(solofpptest)
print('Lenght of data before dropping all wrong group measures: ' + str(lenbefore) + " Len after: " + str(lenafter))

lenbefore = len(solotest)
for items in solotestmatchestodrop:
    solotest = solotest[solotest.matchId != items]
lenafter = len(solotest)
print('Lenght of data before dropping all wrong group measures: ' + str(lenbefore) + " Len after: " + str(lenafter))

In [ ]:
Idtouse = solotest['Id']
solotest = solotest[colls].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
solofpptest = solofpptest[colls].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
tesetdf = tesetdf[colls].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
testoutput = model.predict(solotest)
solotest['Id'] = Idtouse
solotest['winPlacePerc'] = testoutput

In [ ]:
solotest[['Id','winPlacePerc']]

### Train model for submission

In [ ]:
legal_model_output = model.predict(tesetdf)

In [ ]:
#tesetdf = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')
legalmodelsub = rawtest
rawtest['winPlacePerc'] = legal_model_output
submissionlegalmoves = rawtest[['Id','winPlacePerc']]
#submissionlegalmoves
submissionlegalmoves.to_csv("submission.csv",index = False)

In [ ]:
submissionlegalmoves

In [ ]:
submissionlegal_full_games = solotest[['Id','winPlacePerc']]
submissionlegal_full_games
#submissionlegal_full_games.to_csv("submission.csv",index = False)
#submissionlegal_full_games

Example of what the submission needs to look like


In [ ]:
submissiondf = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv')
submissiondf

# Conclusion
The network works fairly well, it is able to predict in such a manner that the difference between the prediction and acutal number can be foreseen. There are some outliers but these seem merely a formality

## 6.2 Model for all matches
### Running on full dataset, without taking expections out

In [ ]:
#traindata = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv')

In [ ]:
#traindata = traindata.dropna()

In [ ]:
#y_train2 = traindata['winPlacePerc']
#X_train2 = traindata[['assists', 'boosts', 'damageDealt',
#       'headshotKills', 'heals', 'killPoints', 'kills',
#       'killStreaks', 'longestKill', 'matchDuration',
#       'rankPoints', 'rideDistance', 'roadKills',
#       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
#       'weaponsAcquired', 'winPoints']].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
#model_fulldata = tf.keras.Sequential(
#    [
#        tf.keras.layers.Dense(300, activation="relu", name="Input_layer"),
#        tf.keras.layers.Dense(100, activation="relu"),
#        tf.keras.layers.Dense(100, activation="relu"),
#        tf.keras.layers.Dense(80, activation="relu"),
#        tf.keras.layers.Dense(40, activation="relu"),
#        tf.keras.layers.Dense(30, activation="relu"),
#        tf.keras.layers.Dense(20, activation="relu"),
#        tf.keras.layers.Dense(1, activation="sigmoid"),
#    ]
#)

#model_fulldata.compile(
#    optimizer=tf.keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
#    loss='mse',
    # List of metrics to monitor
#    metrics=['MSE'],
#)
#history  = model_fulldata.fit(X_train2,y_train2,epochs = 20)

## Submitting the full dataset

In [ ]:
#modeloutputfull_data[0]

In [ ]:
#tesetdf.head(1)

In [ ]:
#Idtouse = tesetdf['Id']
#tesetdf
#tesetdf = tesetdf[colls].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
#modeloutputfull_data = model_fulldata.predict(tesetdf)

In [ ]:
#tesetdf['Id'] = Idtouse
#tesetdf['winPlacePerc'] = modeloutputfull_data

In [ ]:
#tesetdf

In [ ]:
#submissionfulldata = tesetdf[['Id','winPlacePerc']]
#submissionfulldata

In [ ]:
#submissionfulldata.to_csv("submission.csv",index = False)